In [ ]:
# %%capture
# import os
# if "COLAB_" not in "".join(os.environ.keys()):
#     !pip install unsloth vllm
# else:
#     # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
#     !pip install --no-deps unsloth vllm
# # Install latest Hugging Face for Gemma-3!
# !pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
# %%capture
# import os
# if "COLAB_" not in "".join(os.environ.keys()):
#     !pip install unsloth vllm
# else:
#     !pip install --no-deps unsloth vllm
#     # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
#     # Skip restarting message in Colab
#     import sys, re, requests; modules = list(sys.modules.keys())
#     for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
#     !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
#     !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

#     # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
#     f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
#     with open("vllm_requirements.txt", "wb") as file:
#         file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
#     !pip install -r vllm_requirements.txt
# !pip install --upgrade transformers

## **Use this**

In [ ]:
# import os
# import sys
# import re
# import requests

# # Install basic packages
# if "COLAB_" not in "".join(os.environ.keys()):
#     print("Running in Colab environment.")
#     !pip install --no-deps unsloth vllm
# else:
#     print("Running in local environment.")
#     !pip install unsloth vllm

# # Install latest Hugging Face Transformers (special Gemma-3 branch)
# !pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

# # Extra installs for Colab
# if "COLAB_" in "".join(os.environ.keys()):
#     print("Performing extra installations for Colab...")

#     # Reinstall unsloth and vllm (no dependencies)
#     !pip install --no-deps unsloth vllm

#     # Unload conflicting libraries
#     modules = list(sys.modules.keys())
#     for x in modules:
#         if "PIL" in x or "google" in x:
#             sys.modules.pop(x)

#     # Install additional libraries
#     !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
#     !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

#     # Prepare vLLM requirements (excluding numpy, transformers, xformers)
#     f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
#     with open("vllm_requirements.txt", "wb") as file:
#         file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))

#     # Install vLLM requirements
#     !pip install -r vllm_requirements.txt

# print("Setup complete ✅")


In [ ]:
# %%capture
# Install the required libraries
%pip install --upgrade pip
%pip install uv
!uv pip install --system --no-progress --link-mode=symlink accelerate unsloth vllm sentencepiece protobuf datasets wandb huggingface_hub kagglehub[hf-datasets] hf_transfer

In [ ]:
try:
    from kaggle_secrets import UserSecretsClient
    wandb_key = UserSecretsClient().get_secret("WANDB_API_KEY")
    
    # Login to Weights & Biases
    import wandb
    wandb.login(key=wandb_key)
    wandb.init(project="gemma3-4b-wiki", name="finetuning on wiki dataset")
    use_wandb = True
except Exception as e:
    print(f"Error setting up wandb: {e}")
    print("Will continue without wandb tracking")
    use_wandb = False


In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
    # token = "hf_...", # use one if using gated models
)

In [ ]:
from datasets import load_dataset
import pandas as pd

prompt = """
### Instruction:
{}
### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

# Formatting function
def formatting_prompts_func(examples):
    instructions = examples["title"]
    responses = examples["text"]
    texts = []
    for instruction, response in zip(instructions, responses):
        text = prompt.format(instruction, response) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = load_dataset("csv", data_files="/kaggle/input/bangla-wikipedia/wiki.csv", split="train[:10%]")
dataset = dataset.map(formatting_prompts_func, batched = True,)
# dataset = dataset[:len(dataset)//10]

In [ ]:
from peft import PeftModelForCausalLM

if not isinstance(model, PeftModelForCausalLM):
    model = FastModel.get_peft_model(
        model,
        r = 64,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj",
                          "embed_tokens", "lm_head"],  
        lora_alpha = 32,
        lora_dropout = 0,  
        bias = "none",    
        use_gradient_checkpointing = "unsloth",
        random_state = 3407,
        use_rslora = True,
        loftq_config = None,
    )
    print("✅ PEFT (LoRA) adapters attached!")
else:
    print("✅ Model already has LoRA adapters attached! Skipping.")


In [ ]:
# @title Set up a training pipeline using the UnslothTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

max_seq_length = 2048

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        report_to = "wandb",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        # save_steps=1,
    ),
)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Start training
try:
    trainer.train()
    training_successful = True
except KeyboardInterrupt:
    training_successful = True
except Exception as e:
    print(f"Error during training: {e}")
    training_successful = False

In [ ]:
output_dir = "output-bengali-wikidata/final"
# Save the model
if training_successful:
    try:
        trainer.save_model(output_dir)
        print("Model saved successfully!")
    except Exception as e:
        print(f"Error saving model: {e}")

# Finish wandb tracking if it was used
if use_wandb:
    wandb.finish()

In [ ]:
# !cp /content/outputs/checkpoint-1500/adapter_model.safetensors /content/drive/MyDrive/

In [ ]:
import os
import huggingface_hub

# Upload the model to Hugging Face Hub only if training was successful
if training_successful and os.path.exists(output_dir):
    try:
        # Get HF token
        try:
            from kaggle_secrets import UserSecretsClient
            hf_token = UserSecretsClient().get_secret("HF_TOKEN")
        except Exception as e:
            print(f"Error getting HF token: {e}")
            print("Please enter your Hugging Face token manually:")
            hf_token = input()
        
        # Login to Hugging Face
        from huggingface_hub import login
        login(token=hf_token)
        
        from huggingface_hub import HfApi
        api = HfApi()
        
        # Create model card with dataset information
        model_card = """
        ---
        language: bn
        license: cc-by-nc-4.0
        datasets:
        - https://www.kaggle.com/datasets/saurabhshahane/bangla-wikipedia
        ---
        
        # Gemma-3-4b Fine-tuned on Bengali Wikipedia
        
        """
        
        with open("model_card.md", "w") as f:
            f.write(model_card)
        
        # Upload the model and model card
        api.upload_folder(
            folder_path=output_dir,
            path_in_repo=".",
            repo_id="Venkat423/Gemma-3-4b-finetuned-bengali-wikidata",
            repo_type="model",
            commit_message="Upload fine-tuned Gemma-3-4b model on Bengali Wiki dataset",
        )
        
        api.upload_file(
            path_or_fileobj="model_card.md",
            path_in_repo="README.md",
            repo_id="Venkat423/Gemma-3-4b-finetuned-bengali-wikidata",
            repo_type="model",
            commit_message="Add model card",
        )
        
        print("Model uploaded successfully to the Hugging Face Hub!")
    except Exception as e:
        print(f"Error uploading model to Hugging Face Hub: {e}")
else:
    print("Model upload skipped - training was not successful or output directory does not exist.")